# TP 4 : Programmation Dynamique
Anaclet Paul  
Bonrepaux Rémi  

### Récupération des données

In [43]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

## Résolution du problème du sac à dos

In [44]:
function solveSacADos(price, weight, capacity)
    nbObjet = length(price)
    C = zeros(nbObjet,capacity)
    for i in 2:nbObjet
        for j in 2:capacity
            if j-weight[i] >= 1
                C[i,j] = max(C[i-1,j], C[i-1,j-weight[i]] + price[i])
            else
                C[i,j] = C[i-1,j]
            end
        end
    end
    
    #TODO renvoyer 
    sol = C
    
    return sol
end

solveSacADos (generic function with 1 method)

In [53]:
#Test
price, weight, capacity = readKnaptxtInstance("test.opb.txt")
sol = solveSacADos(price, weight, capacity)

#=
ListPrices= 42 40 12 25 ;
ListWeights= 7 4 3 5 ;=#

println(sol)
println("Valeur max des objets transporté :")
println(sol[length(price),capacity])

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 40.0 40.0 40.0 40.0 40.0 40.0; 0.0 0.0 0.0 12.0 40.0 40.0 40.0 52.0 52.0 52.0; 0.0 0.0 0.0 12.0 40.0 40.0 40.0 52.0 52.0 65.0]
Valeur max des objets transporté :
65.0


## Résolution du problème du voyageur

In [55]:
function bellman_ford(Mat,ptsDebut,ptsFinal)
    # initialisation vecteur solution 
    taille = size(Mat, 1)
    Sol = zeros(size(Mat, 1)) .+ Inf
    Sol[ptsDebut] = 0
    # Chemin associer au vecteur solution
    Chemin = zeros(taille)

    #Sommet à exploré
    explo = ptsDebut

    #On s'arrete quand on a un chemin qui va au point finale (le chemin pour aller à F != Inf)
    while Sol[ptsFinal] == Inf 
        #On itére sur tout les sommets accèssible depuis le sommet à explorer
        for i in 1:taille
            if i != explo
                minimum = min(Sol[explo] + Mat[explo,i], Sol[i])
                if minimum != Sol[i]
                    Chemin[i] = explo
                end
                Sol[i] = minimum
            end
            
        end

        #On change le sommet à explorer
        explo = (explo + 1)
        if explo%taille == 0
            explo = explo % taille + 1
        end
    end


    #Remonter pour connaitre le chemin
    Chemin = Int.(Chemin)
    explo = Chemin[ptsFinal] #point finale
    CheminStr = string(ptsFinal)
    while explo != 0
        CheminStr = string(explo, " -> ",CheminStr )
        explo = Chemin[explo]
    end
    return Sol, CheminStr
end

bellman_ford (generic function with 1 method)

In [63]:
# Matrice du problème du cours 
CoutChemin = [0 3 Inf Inf 5 Inf;Inf 0 4 Inf Inf Inf; Inf Inf 0 2 Inf Inf; Inf Inf Inf 0 Inf 3;Inf -1 Inf 9 0 Inf;Inf Inf Inf Inf Inf 0]

# A = 1, B = 2 ...
ptsDepart = 1
ptsFin = 6

Sol, CheminStr = bellman_ford(CoutChemin,ptsDepart,ptsFin)


println("Coût du chemin :")
println(Sol[ptsFin])
println("Chemin a prendre:")
println(CheminStr)

#### Amélioration possible:  
On pourrait améliorer la robustesse par exemple vérifier qu'il y a bien un chemin qui mène au point finale.  
On s'arrete quand la valeur pour arriver au point d'arriver n'est plus infinie. Cependant, rien ne garantie que se soit la meilleur.  